In [1]:
import re
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fidel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fidel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv('CriminalDataver2.csv')
# data.columns = ['id',
#  'case_title',
#  'ponente',
#  'year',
#  'month',
#  'decision_date',
#  'gr_no',
#  'case_link',
#  'content',
#  'has_pdf',
#  'views',
#  'downloads',
#  'status',
#  'modified_by',
#  'created_at',
#  'updated_at']

In [3]:
republic_cases = data

In [4]:
# republic_cases = data[data['year']>='1946'].reset_index()
# republic_cases = republic_cases.head(1000)

In [5]:
def non_ascii(s):
    return "".join(i for i in s if ord(i)<128)

def lower(text):
    return text.lower()

def stop_words(text):
    text = text.split()#split tokens to find stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    #join into a string after removing stop words 
    return text 

def clean_html(text):
    html = re.compile(r'<[^>]+>')#regex
    return html.sub(r'',text)

def punct(text):
    token=RegexpTokenizer(r'\w+')#regex
    text = token.tokenize(text)
    text= " ".join(text)
    return text 

In [6]:
# republic_cases['new_desc'] = republic_cases['text'].apply(clean_html)
# republic_cases['new_desc'] = republic_cases['text'].apply(stop_words)
# republic_cases['new_desc'] = republic_cases.new_desc.apply(func = non_ascii)
# republic_cases['new_desc'] = republic_cases.new_desc.apply(func = lower)
# republic_cases['new_desc'] = republic_cases.new_desc.apply(func = stop_words)
# republic_cases['new_desc'] = republic_cases.new_desc.apply(func = punct)
# republic_cases['new_desc'] = republic_cases['new_desc'].astype(str)
republic_cases['text'] = republic_cases['text'].astype(str)
republic_cases = republic_cases[['year','gr_no_x','text']]

In [7]:
#function for list to string
def listToString(s): 
    # initialize an empty string
    str1 = " " 
    return (str1.join(s))

In [8]:
lmtzr = WordNetLemmatizer()
def sentence_separater(text):
    sentence_list = nltk.tokenize.sent_tokenize(text)
    lemmatized = [[lmtzr.lemmatize(word) for word in word_tokenize(s)] for s in sentence_list]
    lemma_str_sentences = [listToString(s) for s in lemmatized]
    return lemma_str_sentences

In [9]:
# new column with paragraph separated into sentences
republic_cases['lemma_separated'] = republic_cases.text.apply(func = sentence_separater)
# republic_cases['lemma_separated'] = republic_cases.new_desc.apply(func = punct)

<ipython-input-9-13317c0a7a1e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  republic_cases['lemma_separated'] = republic_cases.text.apply(func = sentence_separater)


In [10]:
republic_cases['lemma_separated'][15]

['ha filed a petition soliciting the reopening of this case in accordance with the provision of section of No .',
 '.',
 'The reopening of a cause under section is only available when there exists newly discovered evidence .',
 'If the accused or his attorney knew what would be the testimony of the absent witness or could have known it by the exercise of proper diligence , it can not be said that he ha discovered these proof after the trial .',
 'affidavit whatever setting forth the fact upon this point accompanies the petition a is required by the aforesaid section is , of course , evident that the error of the judge of the inferior court , if error it wa , in the manner of administering the oath to the Chinese witness , can not be presented a newly discovered evidence.For the foregoing reason the petition is denied without deciding ( ) whether the judgment ha become final a soon a registered here , thereby making this motion inopportune ; or , in case it ha not become final ( ) , whe

### civil second degree

In [11]:
property_keywords = ['property', 'house', 'real estate','condo','mansion']
sales_keywords = ['sale', 'barter']
succession_keywords = ['succession']
agency_keywords = ['agency','agent']
partnership_keywords = ['partnership','partner'] 

In [12]:
second_civil_keywords = {
    'is_property': property_keywords,
    'is_sales': sales_keywords,
    'is_succession':succession_keywords,
    'is_agency': agency_keywords,
    'is_partnership': partnership_keywords
}

In [13]:
civil_dict_intent = {intent: re.compile('|'.join(keys)) for intent, keys in second_civil_keywords.items()}

In [14]:
def civil_context(cust_input):
    for intent, pattern in civil_dict_intent.items():
        if re.search(pattern, cust_input): 
            return(intent)
    else:
        return('generic')

second_civil_dict_response = {
    'is_property': 'property case',
    'is_sales': 'sales case',
    'is_succession':'succession case',
    'is_agency': 'agency case',
    'is_partnership': 'partnership case'
}

In [15]:
def second_civil_answer(text):
    response = second_civil_dict_response.get(civil_context(text))
    return response

### commercial second degree

In [16]:
# obligations_keywords = []
negotiable_instruments_keywords = ['bank check','promissory']
corporation_keywords = ['corporation','corporate','incorporate','stock holder','investment']

In [17]:
second_commercial_keywords = {
    'is_negotiable_instruments': negotiable_instruments_keywords,
    'is_corporation':corporation_keywords
}

In [18]:
commercial_dict_intent = {intent: re.compile('|'.join(keys)) for intent, keys in second_commercial_keywords.items()}

In [19]:
def commercial_context(cust_input):
    for intent, pattern in commercial_dict_intent.items():
        if re.search(pattern, cust_input): 
            return(intent)
    else:
        return('generic')

second_commercial_dict_response = {
    'is_negotiable_instruments': 'negotiable instrument case',
    'is_corporation': 'corporation case',
}

In [20]:
def second_commercial_answer(text):
    response = second_commercial_dict_response.get(commercial_context(text))
    return response

### criminal second degree

In [22]:
national_security_keywords = ['treason','sovereignty','international law','war','raid','guerilla','terrorist','pirate']
fundamental_laws_keywords = ['arbitrary','domicile','detention','detain','detainee','religious','priest','god','religion','scripture']

public_order_keywords = ['rebellion','insurrection','coup d etat','sedition','quasirecidivism','separatism','harbor','CPP','NPA','rebel']
drugs_keywords = ['drug','shabu','opium','marijuana','possession','laboratory','meth','operation','seized','bust']
public_morals_keywords = ['gambling','lottery','betting','cockfighting','nudity','indecent','immoral','obscenity','obscene','importation']
committed_officers_keywords = ['bribery','corrupt','graft','plunder','fraud','fund','malversation','promise','gift','officer']
against_liberty_keywords = ['kidnapping','kidnap','slavery','child labor','coercion','vexation','ransom','money','guardian','child']
chastity_keywords = ['adultery','concubine','lascivious','seduce','chastity','sexual','rape','intercourse','seduction','abduction']
status_person_keywords = ['marriage','ceremony','bigamy','married','null','spouse','wife','husband','wedding','void']
against_honor_keywords = ['libel','publish','slander','truth','publicity','incrimination','incriminate','defamation','wire tapping','communication']
quasi_offense_keywords = ['negligence','material','reckless','imprudence','accident','injury','damage','hospital','carelessness','destroy']

against_person_keywords = ['parricide','murder','homicide','suicide','mutiliation','hazing','infanticide','abortion','kill','wound']
public_interest_keywords = ['treasury','counterfeiting','falsification','embezzle','auditor','importing','forgery','certificate','perjury','monopoly']
against_property_keywords = ['robbery','theft','checks','estafa','property','mortgage','arson','carnapping','land','swindling']




In [23]:
second_criminal_keywords = {
    'is_national_security': national_security_keywords,
    'is_fundamental_laws':fundamental_laws_keywords,
    'is_public_interest':public_interest_keywords,
    'is_against_person':against_person_keywords,
    'is_against_property':against_property_keywords,
    'is_public_order':public_order_keywords,
    'is_drugs':drugs_keywords,
    'is_public_morals':public_morals_keywords,
    'is_committed_officers':committed_officers_keywords,
    'is_against_liberty':against_liberty_keywords,
    'is_chastity':chastity_keywords,
    'is_status_person':status_person_keywords,
    'is_against_honor':against_honor_keywords,
    'is_quasi_offense':quasi_offense_keywords
}

In [24]:
criminal_dict_intent = {intent: re.compile('|'.join(keys)) for intent, keys in second_criminal_keywords.items()}

In [25]:
def criminal_context(cust_input):
    for intent, pattern in criminal_dict_intent.items():
        if re.search(pattern, cust_input): 
            return(intent)
    else:
        return('generic')

second_criminal_dict_response = {
    'is_national_security': 'national_security',
    'is_fundamental_laws':'fundamental_laws',
    'is_public_interest':'public_interest',
    'is_against_person':'against_person',
    'is_against_property':'against_property',
    'is_public_order':'public_order',
    'is_drugs':'drugs',
    'is_public_morals':'public_morals',
    'is_committed_officers':'committed_officers',
    'is_against_liberty':'against_liberty',
    'is_chastity':'chastity',
    'is_status_person':'status_person',
    'is_against_honor':'against_honor',
    'is_quasi_offense':'quasi_offense'
}

In [26]:
def second_criminal_answer(text):
    response = second_criminal_dict_response.get(criminal_context(text))
    return response

### labor second degree

In [27]:
employment_keywords = ['employ','employment','employee','recruit','recruitment']
work_keywords = ['work','holiday','paternity','maternity','conditions','retire']
benefits_keywords = ['pay','benefit','wage','coverage','insurance']
union_keywords = ['union','collective','association']

In [28]:
second_labor_keywords = {
    'is_employment':employment_keywords,
    'is_work':work_keywords,
    'is_benefit':benefits_keywords,
    'is_union':union_keywords
}

In [29]:
labor_dict_intent = {intent: re.compile('|'.join(keys)) for intent, keys in second_labor_keywords.items()}

In [30]:
def labor_context(cust_input):
    for intent, pattern in labor_dict_intent.items():
        if re.search(pattern, cust_input): 
            return(intent)
    else:
        return('generic')

second_labor_dict_response = {
    'is_employment':'employment case',
    'is_work':'work case',
    'is_benefits':'benefits case',
    'is_union':'union case'
}

In [31]:
def second_labor_answer(text):
    response = second_labor_dict_response.get(labor_context(text))
    return response

### tax second degree

In [32]:
income_tax_keywords = ['income','estate','business','earnings']
indirect_tax_keywords = ['VAT']
excise_tax_keywords = ['excise','valorem','tobacco','alcohol','petroleum','mining']
stamp_tax_keywords = ['documentary','stamp','dividend','certificate','bonds','insurance','receipt','mortgage']
external_tax_keywords = ['customs','tariff']

In [33]:
second_tax_keywords = {
    'is_income_tax':income_tax_keywords,
    'is_indirect_tax':indirect_tax_keywords,
    'is_excise_tax':excise_tax_keywords,
    'is_stamp_tax':stamp_tax_keywords,
    'is_external_tax':external_tax_keywords,
}

In [34]:
tax_dict_intent = {intent: re.compile('|'.join(keys)) for intent, keys in second_tax_keywords.items()}

In [35]:
def tax_context(cust_input):
    for intent, pattern in tax_dict_intent.items():
        if re.search(pattern, cust_input): 
            return(intent)
    else:
        return('generic')

second_tax_dict_response = {
    'is_income_tax':'income tax case',
    'is_indirect_tax':'indirect tax case',
    'is_excise_tax':'excise tax case',
    'is_stamp_tax':'stamp tax case',
    'is_external_tax':'external tax case'
}

In [36]:
def second_tax_answer(text):
    response = second_tax_dict_response.get(tax_context(text))
    return response

### loop through entire dataset

In [37]:
republic_cases['primary'] = ''
republic_cases['secondary'] = ''

<ipython-input-37-d212baf8ded2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  republic_cases['primary'] = ''
<ipython-input-37-d212baf8ded2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  republic_cases['secondary'] = ''


In [39]:
# iter through dataset with case analyzer
for x in range(len(republic_cases['lemma_separated'])):
    #civil analyzer
    second_civil_answer_per_sentences = [second_civil_answer(y) for y in republic_cases['lemma_separated'][x]]
    property_counts = second_civil_answer_per_sentences.count('property case')
    sales_counts = second_civil_answer_per_sentences.count('sales case')
    succession_counts = second_civil_answer_per_sentences.count('succession case')
    agency_counts = second_civil_answer_per_sentences.count('agency case')
    partnership_counts = second_civil_answer_per_sentences.count('partnership case')
    second_civil_total_counts = {
        property_counts:'property case',
        sales_counts: 'sales case',
        succession_counts:'succession case',
        agency_counts:'agency case',
        partnership_counts:'partnership case'
        }
    temp_civil = max(second_civil_total_counts)
    #commercial analyzer
    second_commercial_answer_per_sentences = [second_commercial_answer(y) for y in republic_cases['lemma_separated'][x]]
    negotiable_instruments_counts = second_commercial_answer_per_sentences.count('negotiable instrument case')
    corporation_counts = second_commercial_answer_per_sentences.count('corporation case')
    second_commercial_total_counts = {
        negotiable_instruments_counts:'negotiable instrument case',
        corporation_counts: 'corporation case',
        }
    temp_commercial = max(second_commercial_total_counts)
    #criminal analyzer
    second_criminal_answer_per_sentences = [second_criminal_answer(y) for y in republic_cases['lemma_separated'][x]]
    national_security_counts = second_criminal_answer_per_sentences.count('national_security')
    fundamental_laws_counts = second_criminal_answer_per_sentences.count('fundamental_laws')
    public_interest_counts = second_criminal_answer_per_sentences.count('public_interest')
    against_person_counts = second_criminal_answer_per_sentences.count('against_person')
    against_property_counts = second_criminal_answer_per_sentences.count('against_property')
    
    public_order_counts = second_criminal_answer_per_sentences.count('public_order')
    drugs_counts = second_criminal_answer_per_sentences.count('drugs')
    public_morals_counts = second_criminal_answer_per_sentences.count('public_morals')
    committed_officers_counts = second_criminal_answer_per_sentences.count('committed_officers')
    against_liberty_counts = second_criminal_answer_per_sentences.count('against_liberty')
    chastity_counts = second_criminal_answer_per_sentences.count('chastity')
    status_person_counts = second_criminal_answer_per_sentences.count('status_person')
    against_honor_counts = second_criminal_answer_per_sentences.count('against_honor')
    quasi_offense_counts = second_criminal_answer_per_sentences.count('quasi_offense')
    
    second_criminal_total_counts = {
        national_security_counts:'national security case',
        fundamental_laws_counts:'fundamental laws case',
        public_interest_counts:'public interest case',
        against_person_counts:'against person case',
        against_property_counts:'against property case',
        public_order_counts:'public moral case',
        drugs_counts:'drug case',
        public_morals_counts:'public moral case',
        committed_officers_counts:'case committed by officer',
        against_liberty_counts:'against liberty case',
        chastity_counts:'against chastity case',
        status_person_counts:'against person status case',
        against_honor_counts:'against honor'
    }
    
    temp_criminal = max(second_criminal_total_counts)
    #labor analyzer
    second_labor_answer_per_sentences = [second_labor_answer(y) for y in republic_cases['lemma_separated'][x]]
    employment_counts = second_labor_answer_per_sentences.count('employment case')
    work_counts = second_labor_answer_per_sentences.count('work case')
    benefits_counts = second_labor_answer_per_sentences.count('benefits case')
    union_counts = second_labor_answer_per_sentences.count('union case')
    second_labor_total_counts = {
        employment_counts:'employment case',
        work_counts:'work case',
        benefits_counts:'benefits case',
        union_counts:'union case',
    }
    temp_labor = max(second_labor_total_counts)
    #tax analyzer
    second_tax_answer_per_sentences = [second_tax_answer(y) for y in republic_cases['lemma_separated'][x]]
    income_tax_counts = second_tax_answer_per_sentences.count('income tax case')
    indirect_tax_counts = second_tax_answer_per_sentences.count('indirect tax case')
    excise_tax_counts = second_tax_answer_per_sentences.count('excise tax case')
    stamp_tax_counts = second_tax_answer_per_sentences.count('stamp tax case')
    external_tax_counts = second_tax_answer_per_sentences.count('external tax case')
    second_tax_total_counts = {
        income_tax_counts:'income tax case',
        indirect_tax_counts:'indirect tax case',
        excise_tax_counts:'excise tax case',
        stamp_tax_counts:'stamp tax case',
        external_tax_counts:'external tax case',
    }
    temp_tax = max(second_tax_total_counts)
    #gathered result
    gathered_total_counts={
        temp_criminal:'criminal case',
        }
    result = gathered_total_counts.get(max(gathered_total_counts))
    if max(gathered_total_counts)==0:
        republic_cases['primary'][x]='no case'
        republic_cases['secondary'][x]='no case'
        continue
    else:
        republic_cases['primary'][x]=result
    if result=='criminal case':
        republic_cases['secondary'][x]=second_criminal_total_counts.get(max(second_criminal_total_counts))

<ipython-input-39-a4f6783cd5fe>:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  republic_cases['primary'][x]=result
C:\Users\fidel\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-39-a4f6783cd5fe>:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  republic_cases['secondary'][x]=second_criminal_total_counts

In [40]:
republic_cases = republic_cases[['year','gr_no_x','text','primary','secondary']]

In [41]:
republic_cases.head()

,year,gr_no_x,text,primary,secondary
0,1901,G.R. No. 91,The accused has been charged with the crime of...,criminal case,against liberty case
1,1901,G.R. No. 299,"No. , , I S I O It appears from the record tha...",criminal case,against person case
2,1901,G.R. No. 43,This is an incident respecting nullity of proc...,criminal case,against person status case
3,1901,G.R. No. 390,It appears that this case was instituted in or...,criminal case,public moral case
4,1901,G.R. No. 202,In civil proceedings instituted to obtain a pr...,criminal case,case committed by officer


In [42]:
republic_cases.to_csv("classified_bianca_keywords.csv")